In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier


from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix,roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

# Column descriptions:

# Prediction vs Norwich (23/12/2021)

In [2]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1 (2).csv')
drop=['pred_poss','Day','Date','Round','Prediction','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
df=df.iloc[:18]

df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [5]:
train=df.iloc[:14]
test=df.iloc[14:]

In [6]:
x_train=train.drop(['GF','GA','L','W'],axis=1)
y_train=train[['L','W']]

In [7]:
x_test=test.drop(['GF','GA','L','W'],axis=1)
y_test=test[['L','W']]

In [16]:
y_train_scoring=train[['GF','GA']]
y_test_scoring=test[['GF','GA']]

    These are the inputs that are supposed to be given to the model. Each value of norwich_input represent the respective value

In [24]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

Predicting match results

In [21]:
norwich_input=[[2.033333333,0.9333333333,58,4411,7.051666667,6.963333333,7.070416667,7.343333333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]

In [22]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
lr.predict(norwich_input)

array([[0, 1]], dtype=uint8)

Predicting match scoring

In [20]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)
lr.predict(norwich_input)

array([[1., 0.]])

###    The model predicts Arsenal will win against Norwich with a score of 1-0.


*******************************************************




# Prediction vs Manchester City (1/1/2021)

    - Dropping all predicted data that were used since the model we will train is based on the actual data only.
    
    - Predicted data will only be used as an input for predicting the match results.

In [51]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1 (3).csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#needs to be change every week since there will be new match data
df=df.iloc[:19]

df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

##### To predict match results

In [94]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

##### To predict score results

In [101]:
y_train_scoring=df[['GF','GA']]

##### Making match result predictions

In [118]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

> These are **predicted data** used as **inputs** for the model to predict(taken from excel).

In [81]:
mc_input=[[2.116666667,0.9166666667,57.83333333,4411,7.011666667,6.892916667,7.219166667,7.566666667,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]]

In [106]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(mc_input)
#lr.predict(mc_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

##### Predicting score results

In [107]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----MC')
lr.predict(mc_input)

ARS----MC


array([[2., 0.]])

## The model predicts Arsenal will win against Manchester City with a score of 2-0.